In [1]:
type Card
    rank::Int64
    suit::Char
end

type Deck
    cards::Array{Card}
end

type Hand
    cards::Array{Card}
end


In [2]:
function newDeck()
    deck = Array{Card}(0)
    for r = 2:14
        for s = ['s','h','d','c']
            push!(deck,Card(r,s))
        end
    end
    for i=1:52
        j = rand(1:52)
        deck[i], deck[j] = deck[j], deck[i]
    end
    return deck
end

function rankString(val)
    if val < 11
        return string(val)
    elseif val == 11
        return "Jack"
    elseif val == 12
        return "Queen"
    elseif val == 13
        return "King"
    else
        return "Ace"
    end
end

function emptyHand()
    return Array{Card}(0)
end

function draw!(deck,hand)
    return push!(hand,pop!(deck))
end

draw! (generic function with 1 method)

In [3]:
function evaluateHand(hand)
    flush = false
    flushSuit = nothing
    suits = Dict('s' => 0, 'h' => 0, 'd' => 0, 'c' => 0)
    valArr = []
    for i=1:size(hand,1)
        suits[hand[i].suit] += 1
        push!(valArr, hand[i].rank)
    end
    for suit in keys(suits)
        if suits[suit] >= 5
            flush = true
            flushSuit = suit
            # now check if it is straight flush
            vals = Dict{Int64,Int64}()
            for i=1:size(hand,1)
                card = hand[i]
                if card.suit == flushSuit
                    vals[card.rank] = 1
                end    
            end
            consec = 0
            vals[1] = get(vals, 14, 0) # since ace can be high or low
            for i=14:-1:1
                if get(vals, i, 0) > 0
                    consec += 1
                else
                    consec = 0
                end
                if consec == 5
                    high = i+4
                    text = "Straight Flush, " * rankString(high) * " high"
                    score = (9,high)
                    return (text,score)
                end
            end
        end
    end
    vals = Dict{Int64,Int64}()
    for i=1:size(hand,1)
        card = hand[i]
        vals[card.rank] = get(vals,card.rank,0) + 1
    end
    threes = Array{Int64}(0)
    twos = Array{Int64}(0)
    for key in keys(vals)
        if vals[key] == 4
            ## 4 of a kind
            high = maximum(filter(e->e != key, valArr))
            text = "Four of a Kind, " * rankString(key) * " with " * rankString(high) * " high"
            score = (8, key * 14 + high)
            return (text, score)
        elseif vals[key] == 3
            push!(threes, key)
            push!(twos, key)
        elseif vals[key] == 2
            push!(twos, key)
        end
    end
    if size(threes,1) > 0 && size(twos,1) > 1
        three = maximum(threes)
        two = maximum(filter(e->e != three, twos))
        text = "Full House, " * rankString(three) * " full of " * rankString(two)
        score = (7, three * 14 + two)
        return (text, score)
    end
    if flush
        high = 0
        for i=1:size(hand,1)
            card = hand[i]
            if card.suit == flushSuit
                high = max(high, card.rank)
            end
        end
        text = "Flush, " * rankString(high) * " high"
        score = (6, high)
        return (text, score)
    end
    # check for straight
    consec = 0
    vals[1] = get(vals, 14, 0) # since ace can be high or low
    for i=14:-1:1
        if get(vals, i, 0) > 0
            consec += 1
        else
            consec = 0
        end
        if consec == 5
            high = i+4
            text = "Straight, " * rankString(high) * " high"
            score = (5, high)
            return (text, score)
        end
    end
    if size(threes,1) > 0
        three = threes[1]
        high1 = maximum(filter(e->e != three, valArr))
        high2 = maximum(filter(e->!(e in [three,high1]), valArr))
        text = "Three of a kind, "*rankString(three)*" highs: "*rankString(high1)*","*rankString(high2)
        score = (4, three * 14^2 + high1*14 + high2)
        return (text, score)
    end
    if size(twos,1) > 1
        highPair = maximum(twos)
        lowPair = maximum(filter(e->e != highPair, twos))
        high = maximum(filter(e->!(e in [highPair,lowPair]), valArr))
        text = "Two pair, "*rankString(highPair)*" and "*rankString(lowPair)*", "*rankString(high)*" high"
        score = (3, highPair*14^2 + lowPair*14 + high)
        return (text, score)
    end
    if size(twos,1) == 1
        pair = maximum(twos)
        high1 = maximum(filter(e->e != pair, valArr))
        high2 = maximum(filter(e->!(e in [pair,high1]), valArr))
        high3 = maximum(filter(e->!(e in [pair,high1,high2]), valArr))
        text = "Pair: "*rankString(pair)*" highs: "*rankString(high1)*","*rankString(high2)*","*rankString(high3)
        score = (2, pair*14^3 + high1*14^2 + high2*14 + high3)
        return (text, score)
    end
    # no hand
    high1 = maximum(valArr)
    high2 = maximum(filter(e->e != high1, valArr))
    high3 = maximum(filter(e->!(e in [high1,high2]), valArr))
    high4 = maximum(filter(e->!(e in [high1,high2,high3]), valArr))
    high5 = maximum(filter(e->!(e in [high1,high2,high3,high4]), valArr))
    text = "Highs: "*rankString(high1)*","*rankString(high2)*","*rankString(high3)*","*rankString(high4)*","*rankString(high5)
    score = (1, high1*14^4 + high2*14^3 + high3*14^2 + high4*14 + high5)
    return (text, score)
end



evaluateHand (generic function with 1 method)

In [ ]:
for j=1:1000000
    deck = newDeck()
    hand = emptyHand()
    draw!(deck,hand)
    draw!(deck,hand)
    draw!(deck,hand)
    draw!(deck,hand)
    draw!(deck,hand)
    draw!(deck,hand)
    draw!(deck,hand)
    evaluateHand(hand)
    #println(hand)
end

In [ ]:
evaluateHand(hand)

In [ ]:
hand = [Card(10,'c'), Card(4,'c'), Card(5,'c'), Card(6,'c'), Card(14,'s'),Card(2,'h'),Card(12,'s')]

In [ ]:
println(evaluateHand(hand))

In [ ]:
typeof(a)

In [4]:
using POMDPs
POMDPs.add("SARSOP")
POMDPs.add("QMDP")
POMDPs.add("DiscreteValueIteration")
POMDPs.add("MCTS")
POMDPs.add("POMDPToolbox")
Pkg.add("Distributions")
Pkg.update()

INFO: Cloning SARSOP from https://github.com/JuliaPOMDP/SARSOP.jl


Package already installed
Package already installed
Package already installed
Package already installed
Package already installed


INFO: Cloning QMDP from https://github.com/JuliaPOMDP/QMDP.jl
INFO: Cloning DiscreteValueIteration from https://github.com/JuliaPOMDP/DiscreteValueIteration.jl
INFO: Cloning MCTS from https://github.com/JuliaPOMDP/MCTS.jl
INFO: Cloning POMDPToolbox from https://github.com/JuliaPOMDP/POMDPToolbox.jl
INFO: Nothing to be done
INFO: METADATA is out-of-date — you may not have the latest version of Distributions
INFO: Use `Pkg.update()` to get the latest versions of your packages
INFO: Updating METADATA...
INFO: Updating cache of Blink...
INFO: Updating cache of DataFrames...
INFO: Updating cache of DataFrames...
INFO: Updating GenerativeModels master...
INFO: Updating POMDPBounds master...
INFO: Updating MCTS master...
INFO: Updating QMDP master... 2de58456 → 22845f20
INFO: Updating DiscreteValueIteration master... 66918f0f → c406999c
INFO: Updating POMDPModels master... a13000ac → 68b99e48
INFO: Updating SARSOP master... c9b46a75 → f9d8c63c
INFO: Updating POMDPToolbox master... 657b1984 → 

Updated 2 taps (homebrew/core, homebrew/science).
==> New Formulae
closure-stylesheets
dnsviz
glfw
gst-rtsp-server
homebrew/science/libeemd
homebrew/science/sollya
node@0.10
node@0.12
node@4
node@5
node@6
tomcat@6
tomcat@7
==> Updated Formulae
antigen
arangodb
argyll-cms
artifactory
aws-elasticbeanstalk
aws-sdk-cpp
awscli
babl
baresip
bazel
chromedriver
chruby-fish
closure-compiler
clutter-gst
cmus
coffeescript
conan
cromwell
crystal-lang
datetime-fortran
devd
docker-machine-driver-xhyve
dovecot
dub
efl
exploitdb
ffmpeg
fluent-bit
fonttools
freetds
gimme
git-ftp
git-lfs
glbinding
grafana
grib-api
groonga
gst-libav
harfbuzz
headphones
heroku
highlight
homebank
homebrew/science/astrometry-net
homebrew/science/ceres-solver
homebrew/science/g2o
homebrew/science/hlaminer
homebrew/science/muscle
homebrew/science/nextflow
homebrew/science/openimageio
homebrew/science/pathvisio
homebrew/science/pcl
homebrew/science/pear
homebrew/science/r-gui
homebrew/science/silo
homebrew/science/siril
homebr

INFO: Building MbedTLS


In [6]:
type PokerState 
    handStr::Int64
    oppHandStr::Int64
    pot::Int64 
    round::Int64
    oppBet::Int64 # opponents bet (-1 if no bet yet)
end

type PokerObs
    str::Int64
    oppBet::Int64
end

type PokerPOMDP <: POMDP{PokerState, Symbol, PokerObs} # Note that our MDP is parametarized by the state and the action
    limit::Int64 # max size of the pot
    lowbet::Int64 
    highbet::Int64
    discount_factor::Float64 # disocunt factor
end

function PokerPOMDP(;limit::Int64=20, # size_x
    lowbet::Int64=1, # size_y
    highbet::Int64=4, # size_y
    discount_factor::Float64=0.9)
    return PokerPOMDP(limit, lowbet, highbet, discount_factor)
end

type PokerStateSpace <: AbstractSpace
    states::Vector{PokerState}
end

function POMDPs.states(pomdp::PokerPOMDP)
    s = PokerState[] 
    for str = 1:10, oppstr = 1:10, pot = 1:pomdp.limit, round = 1:4, oppbet = [-1,0,1,4]
        push!(s, PokerState(str,oppstr,pot,round,oppbet))
    end
    return PokerStateSpace(s)
end;

function POMDPs.iterator(space::PokerStateSpace)
    return space.states 
end;

function POMDPs.state_index(pomdp::PokerPOMDP, state::PokerState)
    betind = state.oppBet + 2
    if betind > 3
        betind = 4
    end
    return sub2ind((10, 10, pomdp.limit,round,  4), state.handStr, state.oppHandStr, state.pot, state.round, betind)
end;

# function POMDPs.rand(rng::AbstractRNG, space::PokerStateSpace, s::PokerState)
#     sp = space.states[rand(rng, 1:end)]
#     copy!(s, sp)
#     s
# end;

type PokerActionSpace <: AbstractSpace
    actions::Vector{Symbol}
end

function POMDPs.actions(pomdp::PokerPOMDP)
    acts = [:fold, :call, :bet, :betHigh]
    return PokerActionSpace(acts)
end;

## actions dont change based on state, just a convenience method
POMDPs.actions(pomdp::PokerPOMDP, s::PokerState, as::PokerActionSpace=actions(pomdp)) = as;

function POMDPs.iterator(space::PokerActionSpace)
    return space.actions 
end;

# function POMDPs.rand(rng::AbstractRNG, space::PokerActionSpace, a::Symbol)
#     return space.actions[rand(rng, 1:end)]
# end;
# function POMDPs.rand(rng::AbstractRNG, space::PokerActionSpace)
#     a = PokerAction(:fold) # doesn't actually affect anything
#     return rand(rng, space, a)
# end;

type PokerObsSpace <: AbstractSpace
    obs::Vector{PokerObs}
end

function POMDPs.observations(pomdp::PokerPOMDP)
    o = PokerObs[] 
    for str = 1:10, oppbet = [-1,0,1,4]
        push!(s, PokerObs(str,oppbet))
    end
    return PokerObsSpace(o)
end;

POMDPs.observations(::PokerPOMDP, s::PokerState, obs::TigerObservationSpace=observations(pomdp)) = obs;

function POMDPs.iterator(space::PokerObsSpace)
    return space.obs 
end;

# POMDPs.create_state(mdp::PokerMDP) = PokerState()
# POMDPs.create_action(mdp::PokerMDP) = :call;


type PokerDistribution <: AbstractDistribution
    neighbors::Array{PokerState} # the states s' in the distribution
    probs::Array{Float64} # the probability corresponding to each state s'
    cat::Categorical # this comes from Distributions.jl and is used for sampling
end

function POMDPs.create_transition_distribution(mdp::GridWorld)
    # can have at most five neighbors in grid world
    neighbors =  [GridWorldState(i,i) for i = 1:5]
    probabilities = zeros(5) + 1.0/5.0
    cat = Categorical(5)
    return GridWorldDistribution(neighbors, probabilities, cat)
end;